In [1]:
!pwd

/panda/eedy/ion_los-angle/s1_a064/notebook


In [2]:
#!/usr/bin/env python3
############################################################
# Program is part of MintPy                                #
# Copyright (c) 2013, Zhang Yunjun, Heresh Fattahi         #
# Author: Wang Yidi, May 2024                              #
############################################################


import os
import datetime as dt
import numpy as np
import math
from scipy import interpolate
from mintpy.objects import ionex, timeseries
from mintpy.simulation import iono
from mintpy.utils import readfile, writefile
from scipy.interpolate import RegularGridInterpolator
from tqdm import tqdm
from mintpy import iono_tec
from mintpy.cli import diff, ifgram_inversion, modify_network, reference_point, reference_date
from mintpy.utils import utils as ut

from datetime import datetime
from netCDF4 import Dataset


ERROR 1: PROJ: proj_create_from_database: Open of /home/eedy/tools/mambaforge/envs/insar/share/proj failed


In [3]:
def compute_lat_lon_ipp(geo_file, utc_sec):
    incidenceAngle = readfile.read(geo_file, datasetName='incidenceAngle')[0]
    incidenceAngle = np.squeeze(incidenceAngle)
    incidenceAngle[incidenceAngle == 0] = np.nan

    azimuthAngle = readfile.read(geo_file, datasetName='azimuthAngle')[0]
    azimuthAngle = np.squeeze(azimuthAngle)
    azimuthAngle[azimuthAngle == 0] = np.nan

    latitude = readfile.read(geo_file, datasetName='latitude')[0]
    latitude = np.squeeze(latitude)
    latitude[latitude == 0] = np.nan

    longitude = readfile.read(geo_file, datasetName='longitude')[0]
    longitude = np.squeeze(longitude)
    longitude[longitude == 0] = np.nan
    
    theta = incidenceAngle*np.pi/180
    Re = 6371000
    h_ipp = 450e3
    theta_ipp = np.arcsin(Re*np.sin(theta)/(Re+h_ipp))
    HEADING = azimuthAngle*np.pi/180

    alpha_ipp = theta - theta_ipp

    latitude_pi = latitude *np.pi/180
    longitude_pi = longitude *np.pi/180

    lat_ipp_pi = np.arcsin(np.sin(latitude_pi)*np.cos(alpha_ipp) + np.cos(latitude_pi)*np.sin(alpha_ipp)*np.cos(HEADING))
    atan2_func = np.vectorize(math.atan2)
    delta = atan2_func(-np.sin(alpha_ipp)*np.cos(latitude_pi)*np.sin(HEADING), np.cos(alpha_ipp) - np.sin(latitude_pi)*np.sin(lat_ipp_pi))
    lon_ipp_pi = np.mod(longitude_pi + delta + np.pi, 2*np.pi) - np.pi

    lat_ipp = lat_ipp_pi *180/np.pi
    lon_ipp = lon_ipp_pi *180/np.pi

    return lat_ipp, lon_ipp ,theta_ipp

def read_netcdf_file(filename):
    # Open the NetCDF file for reading
    dataset = Dataset(filename, 'r')

    # Get the data variables
    varepochs_data = dataset.variables['varepochs'][:]
    time_data = dataset.variables['time'][:]
    lats = dataset.variables['lat'][:]
    lons = dataset.variables['lon'][:]
    tec_maps = dataset.variables['tecmap'][:]
    tec_flag = dataset.variables['tecflag'][:]

    # Close the NetCDF file
    dataset.close()

    mins = []
    for varepoch in varepochs_data:
        dt = datetime.strptime(varepoch, "%Y/%m/%d_%H:%M:%S")
        total_minutes = dt.hour * 60 + dt.minute + dt.second / 60
        mins.append(total_minutes)
    
    # Return the data
    return varepochs_data, time_data, lats, lons, tec_maps, tec_flag , mins

def compute_r_iono(utc_sec, tec_file, lat_ipp, lon_ipp , theta_ipp, geo_file,times_path):
    
    varepochs, time, lats, lons, tec_maps, tecflag , mins= read_netcdf_file(tec_file)
    
    minutes1 = utc_sec / 60
    for i in range(len(mins) - 1):
        if minutes1 >= mins[i] and minutes1 <= mins[i+1]:
            break

    valid_mask = ~(np.isnan(lat_ipp) | np.isnan(lon_ipp))
    valid_lat_ipp = lat_ipp[valid_mask]
    valid_lon_ipp = lon_ipp[valid_mask]

    interp_func = RegularGridInterpolator((mins, lats, lons), tec_maps, method='linear')

    Ei = np.column_stack((np.full(valid_lat_ipp.size, mins[i]),
                          valid_lat_ipp,
                          valid_lon_ipp + (minutes1 - mins[i]) * 360. / (24. * 60.)))

    Ei1 = np.column_stack((np.full(valid_lat_ipp.size, mins[i+1]),
                           valid_lat_ipp,
                           valid_lon_ipp + (minutes1 - mins[i+1]) * 360. / (24. * 60.)))

    new_tec_map1 = np.full_like(lat_ipp, np.nan)
    new_tec_map1[valid_mask] = ((mins[i+1] - minutes1) / (mins[i+1] - mins[i]) * interp_func(Ei) + (minutes1 - mins[i]) / (mins[i+1] - mins[i]) * interp_func(Ei1))

    k = 40.31
    c = 299792458
    meta = readfile.read_attribute(times_path)
    freq = c / float(meta['WAVELENGTH'])
    h_ipp = 450e3
    Re = 6371000
   
    VTEC = new_tec_map1*1e16
    a = VTEC * k / (freq ** 2)
    r_iono = a / np.cos(np.arcsin(np.sin(theta_ipp) / (1 + a)))
    
    return r_iono

def create_iono_timeseries(times_path, tec_dir, geo_file, iono_file):
    
    # download
    date_list = timeseries(times_path).get_date_list()
    tec_files = [f"{tec_dir}/jpld{datetime.strptime(date, '%Y%m%d').timetuple().tm_yday:03d}0.{date[2:4]}i.nc" for date in date_list]
    
    # run
    meta = readfile.read_attribute(times_path)
    utc_sec = float(meta['CENTER_LINE_UTC'])
    lat_ipp, lon_ipp ,theta_ipp = compute_lat_lon_ipp(geo_file, utc_sec)
    
    # write
    num_files = len(tec_files)
    meta = readfile.read_attribute(geo_file)
    width = int(meta['WIDTH'])
    length = int(meta['LENGTH'])
    r_iono = np.zeros((num_files, length,width), dtype=np.float32)
    
    for i in tqdm(range(num_files)):
        try:
            r_iono[i,:,:] = compute_r_iono(utc_sec, tec_files[i], lat_ipp, lon_ipp , theta_ipp, geo_file,times_path)
        except ValueError as e:
            print(f"Encountered a ValueError: {e}")
            print("Skipping to the next iteration...")
            continue

    meta = readfile.read_attribute(times_path)
    ref = meta['REF_DATE']
    
    for i_date_ion in range(len(date_list) - 1):
        if ref == date_list[i_date_ion]: 
            break
            
    r_iono[:,:,:] = r_iono[:,:,:] - r_iono[i_date_ion,:,:]

    ref_y = int(meta['REF_Y'])
    ref_x = int(meta['REF_X'])
    r_iono[:,:,:] = r_iono[:,:,:] - r_iono[:,ref_y,ref_x][:, None, None]
    
    # prepare meta
    meta = readfile.read_attribute(times_path)
    meta['FILE_TYPE'] = 'timeseries'
    meta['UNIT'] = 'm'
    # absolute delay without double reference
    #for key in ['REF_X','REF_Y','REF_LAT','REF_LON','REF_DATE']:
        #if key in meta.keys():
            #meta.pop(key)

    ds_dict = {}
    ds_dict['date'] = np.array(date_list, dtype=np.string_)
    ds_dict['timeseries'] = r_iono
    
    writefile.write(ds_dict, iono_file, metadata=meta)
    return iono_file


dis_file = '../mintpy_hight_gim_ion_upsample/timeseries.h5'
tec_dir =  '/home/eedy/data/aux/IONEX'
geo_file = '../mintpy_hight_gim_ion_upsample/inputs/geometryRadar.h5'
iono_file = '../mintpy_hight_gim_ion_upsample/ion.h5'

ion = create_iono_timeseries(dis_file, tec_dir, geo_file, iono_file)


 88%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▌               | 257/293 [02:23<00:14,  2.44it/s]

Encountered a ValueError: The points in dimension 0 must be strictly ascending or descending
Skipping to the next iteration...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 293/293 [02:39<00:00,  1.84it/s]


delete exsited file: ../mintpy_hight_gim_ion_upsample/ion.h5
create HDF5 file: ../mintpy_hight_gim_ion_upsample/ion.h5 with w mode
create dataset /date       of |S8        in size of (293,)               with compression=None
create dataset /timeseries of float32    in size of (293, 690, 229)      with compression=None
finished writing to ../mintpy_hight_gim_ion_upsample/ion.h5


In [4]:
!diff.py ../mintpy_hight_gim_ion_upsample/timeseries_SET.h5 ../mintpy_hight_gim_ion_upsample/ion.h5 -o ../mintpy_hight_gim_ion_upsample/timeseries_SET_ion.h5 --force

../mintpy_hight_gim_ion_upsample/timeseries_SET.h5 - ['../mintpy_hight_gim_ion_upsample/ion.h5'] --> ../mintpy_hight_gim_ion_upsample/timeseries_SET_ion.h5
the 1st input file is: timeseries
--------------------------------------------------
grab metadata from ref_file: ../mintpy_hight_gim_ion_upsample/timeseries_SET.h5
grab dataset structure from ref_file: ../mintpy_hight_gim_ion_upsample/timeseries_SET.h5
create HDF5 file: ../mintpy_hight_gim_ion_upsample/timeseries_SET_ion.h5 with w mode
create dataset  : bperp      of float32                   in size of (293,)               with compression = None
create dataset  : date       of |S8                       in size of (293,)               with compression = None
create dataset  : timeseries of float32                   in size of (293, 690, 229)      with compression = None
close  HDF5 file: ../mintpy_hight_gim_ion_upsample/timeseries_SET_ion.h5
read from file: ../mintpy_hight_gim_ion_upsample/ion.h5
read from file: ../mintpy_hight_gi